In [ ]:


clip = CTCLIP(
    image_encoder = image_encoder,
    text_encoder = text_encoder,
    dim_image = 2097152,
    dim_text = 768,
    dim_latent = 512,
    extra_latent_projection = False,         # whether to use separate projections for text-to-image vs image-to-text comparisons (CLOOB)
    use_mlm = False,
    downsample_image_embeds = False,
    use_all_token_embeds = False

)

#Load the pretrained weights during inference

clip.load("C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\CT-CLIP\CT_VocabFine_v2.pt")

In [1]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [ ]:
file_path = r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\TEST CT SCANS"

In [4]:
import os
import pydicom
import numpy as np
import torch
from scipy.ndimage import zoom
from torchvision import transforms as T

def preprocess_dicom_directory(directory, image_size=256):
    """
    Load DICOM files from a directory, stack into a 3D volume, and preprocess for CT-CLIP.
    
    Args:
        directory (str): Path to directory containing .dcm files.
        image_size (int): Target size for resizing (default: 256).
    
    Returns:
        torch.Tensor: Preprocessed CT volume with shape (1, 3, image_size, image_size, image_size).
    """
    # Read all .dcm files from the directory
    dicom_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.dcm')]
    dicom_files.sort()  # Ensure slices are in order (assuming filename sorting is sufficient)

    # Load DICOM slices
    slices = [pydicom.dcmread(f).pixel_array for f in dicom_files]
    volume = np.stack(slices, axis=-1)  # Shape: (height, width, depth)

    # Normalize pixel values (HU to [0, 1] or model-specific range)
    volume = (volume - np.min(volume)) / (np.max(volume) - np.min(volume) + 1e-6)

    # Resize to target size (image_size, image_size, image_size)
    zoom_factors = (image_size / volume.shape[0], image_size / volume.shape[1], image_size / volume.shape[2])
    volume = zoom(volume, zoom_factors, order=1)  # Linear interpolation

    # Convert to tensor and add channel dimension
    volume = torch.tensor(volume, dtype=torch.float32)
    if volume.ndim == 3:
        volume = volume.unsqueeze(0)  # Add channel dim: (1, H, W, D)
        volume = volume.repeat(3, 1, 1, 1)  # Repeat to simulate 3 channels: (3, H, W, D)
    else:
        volume = volume.permute(3, 0, 1, 2)  # (H, W, D, C) -> (C, H, W, D)

    # Apply normalization (as defined in visual_ssl.py)
    normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    volume = normalize(volume)

    # Add batch dimension
    volume = volume.unsqueeze(0)  # Shape: (1, 3, image_size, image_size, image_size)
    return volume



In [14]:
import os
import torch
from ct_clip.ct_clip import CTCLIP, VisionTransformer
from transformers import BertModel, BertTokenizer

# Define model parameters
dim_text = 768  # Matches BiomedVLP-CXR-BERT-specialized output dimension
dim_image = 294912  # Adjust based on VisionTransformer output
dim_latent = 512
num_text_tokens = 28897
text_seq_len = 512
visual_image_size = 256
visual_patch_size = 32  # Adjust if needed (see below)
channels = 3

# Initialize text transformer using BertModel
text_transformer = BertModel.from_pretrained('microsoft/BiomedVLP-CXR-BERT-specialized')

# Initialize vision transformer
# The shape mismatch for to_visual_latent.weight ([512, 294912]) suggests a larger output dimension
# Calculate expected dim_image: 294912 / (256 // 32) ** 2 = 294912 / 64 = 4608 tokens
# This implies a different patch size or additional features
vision_transformer = VisionTransformer(
    dim=dim_image,
    image_size=visual_image_size,
    patch_size=visual_patch_size,  # May need adjustment (e.g., smaller patch size)
    channels=channels,
    depth=6,
    heads=8,
    dim_head=64,
    patch_dropout=0.5
)

# Initialize CT-CLIP
model = CTCLIP(
    image_encoder=vision_transformer,
    text_encoder=text_transformer,
    dim_text=dim_text,
    dim_image=dim_image,
    dim_latent=dim_latent,
    num_text_tokens=num_text_tokens,
    text_seq_len=text_seq_len,
    visual_image_size=visual_image_size,
    visual_patch_size=visual_patch_size,
    channels=channels,
    use_mlm=False,
    use_visual_ssl=False,
    visual_ssl_type='simsiam',
    text_ssl_loss_weight=0.05,
    image_ssl_loss_weight=0.05,
    multiview_loss_weight=0.1
)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load pretrained weights
state_dict = torch.load(
    r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\CT-CLIP\CT_VocabFine_v2.pt",
    map_location=device
)
model.load_state_dict(state_dict, strict=False)  # Use strict=False to ignore missing/unexpected keys

You are using a model of type cxr-bert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at microsoft/BiomedVLP-CXR-BERT-specialized and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3623878656 bytes.